In [1]:
import re
import pandas as pd
import numpy as np
from scipy.stats import spearmanr, pearsonr
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
df_str_rel = pd.read_csv('C:\\Users\\ADMIN\\Downloads\\eng_train (1).csv')
df_str_rel.head()

,PairID,Text,Score
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0
3,ENG-train-0003,If he is good looking and has a good personali...,1.0
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0


In [3]:
df_str_rel['Split_Text'] = df_str_rel['Text'].apply(lambda x: x.split("\n"))
df_str_rel.head()

,PairID,Text,Score,Split_Text
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"[It that happens, just pull the plug., if that..."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,"[A black dog running through water., A black d..."
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,"[I've been searchingthe entire abbey for you.,..."
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,[If he is good looking and has a good personal...
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"[She does not hate you, she is just annoyed wi..."


In [4]:
from transformers import AutoModel, AutoTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Specify the model name or path
model_name_or_path = "ai4bharat/indic-bert"

# Load the IndicBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModel.from_pretrained(model_name_or_path)

# Function to encode text and get embeddings
def encode_text(text):
    input_ids = tokenizer(text, return_tensors="pt", padding=True, truncation=True)["input_ids"]
    with torch.no_grad():
        embeddings = model(input_ids).last_hidden_state.mean(dim=1)  # Average pooling
    return embeddings

# Calculate cosine similarity between two sentence embeddings
def calculate_similarity(sentence1, sentence2):
    embeddings1 = encode_text(sentence1)
    embeddings2 = encode_text(sentence2)
    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity[0][0]

sentence1 = "This is the first sentence."
sentence2 = "This is the second sentence."

similarity = calculate_similarity(sentence1, sentence2)
print(f"Cosine Similarity: {similarity:.4f}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Cosine Similarity: 0.9665


In [5]:
true_scores = df_str_rel['Score'].values
pred_scores = []

for index,row in df_str_rel.iterrows():
  s1,s2 = row["Text"].split("\n")

  # Overlap score
  pred_scores.append(calculate_similarity(s1,s2))

In [6]:
print("Spearman Correlation:", round(spearmanr(true_scores,pred_scores)[0],2))

Spearman Correlation: 0.42
